In [1]:
!pip install torch_geometric
!pip install wandb
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.1+cu121.html
!pip install umap-learn
!pip install hdbscan
import wandb
import torchvision
import torch
from torch.utils.data import Dataset
from torchvision import datasets
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from PIL import Image
from numpy.linalg import norm
from torch.utils.data import random_split
import torch.optim as optim
import torchvision.transforms as transforms
import math
#GNN stuff
import torch_geometric
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import umap
import hdbscan
print(device)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.5 MB/s eta 0:00:00a 0:00:01
Looking in links: https://data.pyg.org/whl/torch-2.1.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 58.0 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 78.6 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 75.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.0/936.0 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 17.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 39.0 MB/s eta 0:00:00:00:0100:

2024-08-29 14:51:41.380154: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-29 14:51:41.380243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-29 14:51:41.382031: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-29 14:51:41.392060: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 14:51:42.888993: W tensorflow/compiler/tf2

cuda:0


Metrics


In [2]:
import numpy as np
from sklearn.metrics.cluster import contingency_matrix


def precision(y_true, y_pred):
  true_positives, false_positives, _, _ = _compute_counts(y_true, y_pred)
  return true_positives / (true_positives + false_positives)


def recall(y_true, y_pred):
  true_positives, _, false_negatives, _ = _compute_counts(y_true, y_pred)
  return true_positives / (true_positives + false_negatives)


def accuracy_score(y_true, y_pred):
  true_positives, false_positives, false_negatives, true_negatives = _compute_counts(
      y_true, y_pred)
  return (true_positives + true_negatives) / (
      true_positives + false_positives + false_negatives + true_negatives)


def _compute_counts(y_true, y_pred):  # TODO(tsitsulin): add docstring pylint: disable=missing-function-docstring
  contingency = contingency_matrix(y_true, y_pred)
  same_class_true = np.max(contingency, 1)
  same_class_pred = np.max(contingency, 0)
  diff_class_true = contingency.sum(axis=1) - same_class_true
  diff_class_pred = contingency.sum(axis=0) - same_class_pred
  total = contingency.sum()

  true_positives = (same_class_true * (same_class_true - 1)).sum()
  false_positives = (diff_class_true * same_class_true * 2).sum()
  false_negatives = (diff_class_pred * same_class_pred * 2).sum()
  true_negatives = total * (
      total - 1) - true_positives - false_positives - false_negatives

  return true_positives, false_positives, false_negatives, true_negatives



Load High dimen MNIST

In [3]:
import torch
dataset = np.load('/notebooks/mnist.npy')
#dataset = np.load('/notebooks/70ktransformeddataset.npy')
file_path = '/notebooks/mnist.gt'
#dataset = np.load('/notebooks/cifar10.npy')
#dataset = np.load('/notebooks/70ktransformeddataset.npy')
#dataset = np.load('HighDimenData/birds.npy')
#file_path = 'HighDimenData/birds.gt'
#dataset = np.load('HighDimenData/imagenet.npy')
#file_path = 'HighDimenData/imagenet.gt'
labels = []
with open(file_path, 'r') as file:
    for line in file:
        labels.append(int(line.strip()))
#labels = np.load('/notebooks/cifar10labels.npy')
labels= torch.tensor(labels)
dataset = torch.from_numpy(dataset)
print(dataset.size())

torch.Size([70000, 784])


Neighbor graph gen

In [4]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
#5
dataset_flat = dataset.reshape(dataset.shape[0], -1)
k = 50
nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(dataset_flat)
distances, indices = nbrs.kneighbors(dataset_flat)

# Calculate inverse distances, with a small epsilon to avoid division by zero
epsilon = 1e-5
inverse_distances = 1 / (distances.flatten() + epsilon)

num_nodes = indices.shape[0]
source_nodes = np.repeat(np.arange(num_nodes), k)
target_nodes = indices.flatten()
edge_index = np.vstack([source_nodes, target_nodes])
#np.save('edgeindex70k.npy',edge_index)
edge_index = torch.from_numpy(edge_index).long()
edge_weight = torch.from_numpy(inverse_distances).float()
#np.save('edgeweight70k.npy',edge_weight)

In [5]:
train_mask = np.zeros(num_nodes, dtype=bool)
train_mask[:70000] = True
 
# Creating the test mask
test_mask = np.zeros(num_nodes, dtype=bool)
test_mask[20000:] = True

train_mask = torch.from_numpy(train_mask)
test_mask = torch.from_numpy(test_mask)

UMAP Dimensionality Reduction

Planar Graph Gen

In [6]:
from torch_geometric.loader import NeighborLoader
from torch_geometric.loader import RandomNodeLoader
from torch_geometric.data import Data
data = Data(x=dataset, edge_index=edge_index, edge_weight = edge_weight, y = labels, train_mask = train_mask, test_mask = test_mask)
#30
train_loader = NeighborLoader(data, input_nodes = data.train_mask, num_neighbors=[30]*2, shuffle=True, num_workers = 2, batch_size =16)
test_loader = NeighborLoader(data,input_nodes= data.test_mask, num_neighbors=[-1], shuffle=False, num_workers = 2,batch_size = 32)
sampled_hetero_data = next(iter(train_loader))
print(sampled_hetero_data.y)

tensor([8, 1, 1,  ..., 5, 5, 5])


In [17]:


class FireModule(nn.Module):
    def __init__(self, in_channels, squeeze_channels, expand1x1_channels, expand3x3_channels):
        super(FireModule, self).__init__()
        self.squeeze = nn.Conv2d(in_channels, squeeze_channels, kernel_size=1)
        self.expand1x1 = nn.Conv2d(squeeze_channels, expand1x1_channels, kernel_size=1)
        self.expand3x3 = nn.Conv2d(squeeze_channels, expand3x3_channels, kernel_size=3, padding=1)
    
    def forward(self, x):
        x = self.squeeze(x)
        x = F.relu(x)
        return torch.cat([
            self.expand1x1(x),
            self.expand3x3(x)
        ], 1)

class SqueezeNet(nn.Module):
    def __init__(self):
        super(SqueezeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.fire1 = FireModule(32, 8, 16, 16)
        self.fire2 = FireModule(32, 8, 16, 16)
        self.fire3 = FireModule(32, 16, 32, 32)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv_final = nn.Conv2d(64, 128, kernel_size=1)  # 128 output channels

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)  # Reshape to 28x28 images with 1 channel
        x = F.relu(self.conv1(x))
        x = self.pool(self.fire1(x))
        x = self.pool(self.fire2(x))
        x = self.pool(self.fire3(x))
        x = F.relu(self.conv_final(x))
        x = F.adaptive_avg_pool2d(x, (1, 1))  # Reduce to [batch_size, 128, 1, 1]
        x = x.view(-1, 128)  # Flatten to [batch_size, 128]
        return x

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe360787350>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fe48bab2990, raw_cell="

class FireModule(nn.Module):
    def __init__(se.." store_history=True silent=False shell_futures=True cell_id=f891d4fb-6874-417b-aca5-cec3203fbc5e>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe360787350>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fe3e0b1fbd0, execution_count=17 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fe48bab2990, raw_cell="

class FireModule(nn.Module):
    def __init__(se.." store_history=True silent=False shell_futures=True cell_id=f891d4fb-6874-417b-aca5-cec3203fbc5e> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Input: (3, 32, 32) -> Output: (64, 32, 32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Input: (64, 32, 32) -> Output: (128, 32, 32)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # Input: (128, 16, 16) -> Output: (256, 16, 16)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Reduces each dimension by half

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 8 * 8, 512)  # Adjusted based on the output from conv layers
        self.fc2 = nn.Linear(512, 128)  # Output 128 features for GNN input

    def forward(self, x):
        x = F.relu(self.conv1(x))  # After conv1: (64, 32, 32)
        x = self.pool(F.relu(self.conv2(x)))  # After conv2 + pool: (128, 16, 16)
        x = self.pool(F.relu(self.conv3(x)))  # After conv3 + pool: (256, 8, 8)
        x = x.view(-1, 128 * 8 * 8)  # Flatten the tensor for the fully connected layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [9]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64* 7 * 7, 256)  # Flattening to 128 features

    def forward(self, x):
        x = x.view(-1, 1, 28, 28).to(device)  # Reshape to 28x28 images with 1 channel
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        return x


In [19]:
import torch
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU, Dropout
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_mean_pool, GCNConv
from torch_geometric.data import Batch
from torch_geometric.nn import DenseGraphConv, GCNConv
from torch_geometric.utils import to_dense_adj

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout_rate=0.5):
        super(GCN, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()
        self.dropout_rate = dropout_rate
        self.pool = DMoNPooling([hidden_channels, hidden_channels], out_channels)
        # Initial GAT layer
        self.convs.append(GCNConv(in_channels, hidden_channels, add_self_loops=True))
        self.batch_norms.append(BatchNorm1d(hidden_channels))

        # Hidden GAT layers
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels, add_self_loops=True))
            self.batch_norms.append(BatchNorm1d(hidden_channels))

        # Final GAT layer
        self.convs.append(GCNConv(hidden_channels, out_channels, add_self_loops=True))
        self.batch_norms.append(BatchNorm1d(out_channels))

    def forward(self, x, edge_index):
        for conv, batch_norm in zip(self.convs[:-1], self.batch_norms[:-1]):
            x = conv(x, edge_index)
            x = batch_norm(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)
            #print(x.size())

       # x = self.convs[-1](x, edge_index, edge_attr = edge_attr)
        adj = to_dense_adj(edge_index)
        #print(x.size())
        x, _, adj, sp1, o1, c1 = self.pool(x, adj)
        #print(x.size())

        return F.log_softmax(x, dim=-1), sp1 + o1 + c1

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe360787350>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fe4b8577950, raw_cell="import torch
from torch.nn import Sequential, Line.." store_history=True silent=False shell_futures=True cell_id=b67c300d-f005-4551-9b09-28db50010607>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe360787350>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fe48b9627d0, execution_count=19 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fe4b8577950, raw_cell="import torch
from torch.nn import Sequential, Line.." store_history=True silent=False shell_futures=True cell_id=b67c300d-f005-4551-9b09-28db50010607> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [18]:
import torch
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU, Dropout
import torch.nn.functional as F
from torch_geometric.nn import SuperGATConv, global_mean_pool
from torch_geometric.data import Batch
from torch_geometric.nn import DenseGraphConv, DMoNPooling, GCNConv
from torch_geometric.utils import to_dense_adj

class SUPAGAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, num_heads, dropout_rate=0.5):
        super(SUPAGAT, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()
        self.dropout_rate = dropout_rate
        self.pool = DMoNPooling([hidden_channels*num_heads, hidden_channels*num_heads], out_channels)
        # Initial GAT layer
        self.convs.append(SuperGATConv(in_channels, hidden_channels, heads = num_heads,attention_type='MX',edge_sample_ratio=0.8,is_undirected=True))
        self.batch_norms.append(BatchNorm1d(hidden_channels*num_heads))

        # Hidden GAT layers
        for _ in range(num_layers - 2):
            self.convs.append(SuperGATConv(hidden_channels*num_heads, hidden_channels, heads = num_heads,attention_type='MX',edge_sample_ratio=0.8,  is_undirected=True))
            self.batch_norms.append(BatchNorm1d(hidden_channels*num_heads))

        # Final GAT layer
        self.convs.append(SuperGATConv(hidden_channels*num_heads, out_channels, heads = num_heads,attention_type='MX',edge_sample_ratio=0.8,  is_undirected=True))
        self.batch_norms.append(BatchNorm1d(out_channels*num_heads))

    def forward(self, x, edge_index):
        att_loss = 0
        for conv, batch_norm in zip(self.convs[:-1], self.batch_norms[:-1]):
            x = conv(x, edge_index)
            att_loss += conv.get_attention_loss()
            x = batch_norm(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)

       # x = self.convs[-1](x, edge_index, edge_attr = edge_attr)
        adj = to_dense_adj(edge_index)
       # print(x.size())
        x, _, adj, sp1, o1, c1 = self.pool(x, adj)
       # print(x.size())

        return F.log_softmax(x, dim=-1), sp1 + o1 + c1+ att_loss

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe360787350>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fe3b1cba410, raw_cell="import torch
from torch.nn import Sequential, Line.." store_history=True silent=False shell_futures=True cell_id=da5ea194-410a-4606-b9ba-57cd001adc77>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe360787350>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fe47ac51e90, execution_count=18 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fe3b1cba410, raw_cell="import torch
from torch.nn import Sequential, Line.." store_history=True silent=False shell_futures=True cell_id=da5ea194-410a-4606-b9ba-57cd001adc77> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [20]:
class combine(torch.nn.Module):
    def __init__(self, cnn, gnn):
        super(combine, self).__init__()
        self.cnn = cnn
        self.gnn = gnn

    def forward(self, x, edge_index):
        x = self.cnn(x)  
        #print(x.size())
        x, loss = self.gnn(x, edge_index)  # Apply GNN

        return x, loss

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe360787350>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fe361712a10, raw_cell="class combine(torch.nn.Module):
    def __init__(s.." store_history=True silent=False shell_futures=True cell_id=4f4822b9-aba4-44ad-b7dc-582b01db24b4>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe360787350>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fe48b9bec90, execution_count=20 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fe361712a10, raw_cell="class combine(torch.nn.Module):
    def __init__(s.." store_history=True silent=False shell_futures=True cell_id=4f4822b9-aba4-44ad-b7dc-582b01db24b4> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [21]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.optim.lr_scheduler import ReduceLROnPlateau
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#gnn = SUPAGAT(in_channels= 128, hidden_channels=16, out_channels=10, num_heads =8, num_layers=5, dropout_rate=0.3).to(device)
gnn = GCN(in_channels=  128, hidden_channels=64, out_channels=10, num_layers=5, dropout_rate=0.3).to(device)
#cnn = CNN().to(device)
cnn = SqueezeNet().to(device)
#cnn = CIFAR10_CNN().to(device)

model = combine(cnn, gnn)
#model = GIN(in_channels= 128, hidden_channels=128, out_channels=10, num_layers=5, dropout_rate=0.3).to(device)
#model = GCN(in_channels= 784, hidden_channels=625, out_channels=10, num_layers=5, dropout_rate=0.3).to(device)
#model = GCNpool(in_channels= 784, hidden_channels=128, out_channels=10, num_layers=5, dropout_rate=0.3).to(device)
#model = Net(in_channels= 784,out_channels=10,hidden_channels=128).to(device)
#model = HGT(in_channels= 512, hidden_channels = 128, num_heads=2, out_channels=10, num_layers=5, dropout_rate=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience= 5, factor = 0.75)
 

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe360787350>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fe3e0b2afd0, raw_cell="from torch.optim.lr_scheduler import CosineAnneali.." store_history=True silent=False shell_futures=True cell_id=22d5e676-30b3-4a03-809a-23a1d3aec8e0>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe360787350>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fe48b9bce90, execution_count=21 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fe3e0b2afd0, raw_cell="from torch.optim.lr_scheduler import CosineAnneali.." store_history=True silent=False shell_futures=True cell_id=22d5e676-30b3-4a03-809a-23a1d3aec8e0> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [14]:
import torch
import torch.nn as nn

class lossmodel(nn.Module):
    def __init__(self, n_clusters, dropout_rate = 0):
        super(lossmodel, self).__init__()
        self.n_clusters = n_clusters
        self.dropout_rate = 0

        # Define the layers
        self.transform = nn.Sequential(
            nn.Linear(n_clusters, n_clusters),
            nn.Dropout(dropout_rate)
        )

        # Initialize weights
        nn.init.orthogonal_(self.transform[0].weight)
        nn.init.zeros_(self.transform[0].bias)

    def forward(self, x):
        return self.transform(x)


def calculate_collapse_loss(number_of_nodes, n_clusters, output, lossize):
    model = lossmodel(n_clusters).to(device)
    assignments = F.softmax(model(output), dim=1)
    cluster_sizes = torch.sum(assignments, dim=0)
    collapse_loss = torch.norm(cluster_sizes) / number_of_nodes * torch.sqrt(torch.tensor(float(n_clusters))) - 1
    return collapse_loss*lossize

In [22]:
config = {
    "architecture":"squeeze-net-gcn",
    "clusters":10,
    "dataset":"mnist",
    "epochs":50,
    "hidden_channel":16,
    "layers":5,
    "learning_rate":0.001,
    "loss_function":"ALL",
    "neighbors":50,
    "batch-size": 64,
    "umap-size":256,
    "umap-neighbors":50,
    "num-heads": 8, 
    "sampling-ratio" : 0.8
}

# Pass the config dictionary when you initialize W&B
run = wandb.init(project="PRIMES-GNN", config=config, name = 'squeezenetgcn')
#e807d0434ea6864dc92d6c99cdcfa073feaca0cc



Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe360787350>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fe3cd021250, raw_cell="config = {
    "architecture":"squeeze-net-gcn",
 .." store_history=True silent=False shell_futures=True cell_id=8e6314b7-4bdc-4a11-9888-41161462b743>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

acc,▁█
loss,█▁
nmi,▁█
acc,0.73167
loss,2.71178
nmi,0.12911


Neighbor loader with Plateau scheduler

In [ ]:
import  time  
import torch_sparse
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import adjusted_rand_score  
classes = 10
def train():
    model.train()
    loss_all = 0
    count = 0
    for data in train_loader:
        count = count+1
        optimizer.zero_grad()
        data = data.to(device)
        #out,tot_loss = model(data.x, data.edge_index, data.edge_weight)
        out,tot_loss = model(data.x, data.edge_index)
        out = out.squeeze()
        loss = tot_loss + calculate_collapse_loss(data.num_nodes,classes,out,0.1)
        #loss = F.cross_entropy(out, data.y) 
        loss_all += loss.item()
        loss.backward()
        optimizer.step()
    return loss_all/count


@torch.no_grad()
def test(loader):
    model.eval()
    correct = 0
    loss_all = 0
    nmi = 0
    count = 0  
    avgARI = 0
    for data in loader:
        count +=1
        data.to(device)
        #pred,tot_loss = model(data.x, data.edge_index, data.edge_weight)
        pred,tot_loss = model(data.x, data.edge_index)
        pred = pred.squeeze()
        loss = tot_loss+ calculate_collapse_loss(data.num_nodes,classes,pred,0.1  )
        #loss = F.cross_entropy(pred, data.y) 
        loss_all += loss.item()
        correct += accuracy_score(data.y.cpu().detach().numpy(),pred.max(dim=-1)[1].cpu().detach().numpy())
        nmi += normalized_mutual_info_score(data.y.cpu().detach().numpy(),pred.max(dim=-1)[1].cpu().detach().numpy())
        avgARI +=adjusted_rand_score(data.y.cpu().detach().numpy(),pred.max(dim=-1)[1].cpu().detach().numpy())
    return loss_all/count, correct/count, nmi/count, avgARI/count


times = []
best_nmi = 0
for epoch in range(1, 70):
    start = time.time()
    train_loss = train()
    print(time.time()-start)
    #_, train_acc,nmi = test(train_loader)
    train_acc = 0
    print(time.time()-start)
    test_loss, test_acc,nmi, ari = test(test_loader)
    wandb.log({'acc': test_acc, 'loss': test_loss, "nmi": nmi, "ari":ari})
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.3f}, '
          f'Train Acc: {train_acc:.3f},  '
          f' Test Loss: {test_loss:.3f}, '
          f'Test Acc: {test_acc:.3f},'
          f'NMI:{nmi:.3f},'
          f'ARI:{ari:.3f}')
    times.append(time.time() - start)
    scheduler.step(test_loss)
    if nmi > best_nmi:
        torch.save(model.state_dict(), 'gcnmodel_weights.pth')
        best_nmi = nmi

print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

711.7996096611023
711.8003051280975
Epoch: 001, Train Loss: -0.289, Train Acc: 0.000,   Test Loss: 0.087, Test Acc: 0.921,NMI:0.652,ARI:0.588
737.4336223602295
737.4343955516815
Epoch: 002, Train Loss: -0.366, Train Acc: 0.000,   Test Loss: 0.049, Test Acc: 0.926,NMI:0.693,ARI:0.629
733.6329205036163
733.6336350440979
Epoch: 003, Train Loss: -0.404, Train Acc: 0.000,   Test Loss: 0.083, Test Acc: 0.925,NMI:0.707,ARI:0.634
739.8320560455322
739.8328478336334
Epoch: 004, Train Loss: -0.422, Train Acc: 0.000,   Test Loss: 0.052, Test Acc: 0.933,NMI:0.732,ARI:0.670
767.8897695541382
767.8905045986176
Epoch: 005, Train Loss: -0.430, Train Acc: 0.000,   Test Loss: 0.086, Test Acc: 0.925,NMI:0.708,ARI:0.629
783.2146277427673
783.2152409553528
Epoch: 006, Train Loss: -0.435, Train Acc: 0.000,   Test Loss: 0.072, Test Acc: 0.929,NMI:0.713,ARI:0.649
811.1472804546356
811.1479165554047
Epoch: 007, Train Loss: -0.440, Train Acc: 0.000,   Test Loss: 0.047, Test Acc: 0.933,NMI:0.734,ARI:0.680
791.98

In [ ]:
import time   
import torch_sparse
from sklearn.metrics.cluster import nor malized_mutual_info_score
classes = 10
def train():
    model.train()
    loss_all = 0
    count = 0
    for data in train_loader:
        count = count+1
        optimizer.zero_grad()
        data = data.to(device)
        x_dict = {
            'node': data['node'].x
        }
        edge_index_dict = {
            ('node', 'to', 'node'): data['node', 'to', 'node'].edge_index

        }
        out,tot_loss = model(x_dict,edge_index_dict)
        out = out.squeeze()
        #loss = tot_loss + calculate_collapse_loss(data.num_nodes,classes,out,0.1)
        labels = data['node'].y
        loss = F.cross_entropy(out, labels)
        loss_all += loss.item()
        loss.backward()
        optimizer.step()
    return loss_all/count


@torch.no_grad()
def test(loader):
    model.eval()
    correct = 0
    loss_all = 0
    nmi = 0
    count = 0
    for data in loader:
        count +=1
        data.to(device)
        x_dict = {
            'node': data['node'].x
        }
        edge_index_dict = {
            ('node', 'to', 'node'): data['node', 'to', 'node'].edge_index

        }
        pred,tot_loss = model(x_dict,edge_index_dict)
        pred = pred.squeeze()
        #loss = tot_loss+ calculate_collapse_loss(data.num_nodes,classes,pred,0.1)
        labels = data['node'].y
        loss = F.nll_loss(pred, labels)
        labels = data['node'].y.cpu().detach().numpy()
        loss_all += loss.item()
        correct += accuracy_score(labels,pred.max(dim=-1)[1].cpu().detach().numpy())
        nmi += normalized_mutual_info_score(labels,pred.max(dim=-1)[1].cpu().detach().numpy())
    return loss_all/count, correct/count, nmi/count


times = []
best_nmi = 0
for epoch in range(1, 70):
    start = time.time()
    train_loss = train()
    print(time.time()-start)
    _, train_acc,nmi = test(train_loader)
    print(time.time()-start)
    test_loss, test_acc,nmi = test(test_loader)
    wandb.log({'acc': test_acc, 'loss': test_loss, "nmi": nmi})
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.3f}, '
          f'Train Acc: {train_acc:.3f},  '
          f' Test Loss: {test_loss:.3f}, '
          f'Test Acc: {test_acc:.3f}')
    times.append(time.time() - start)
    scheduler.step(test_loss)

print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import adjusted_rand_score
def metrics():
    model.eval()
    avgacc = 0
    avgARI = 0
    loss_all = 0
    avgprec = 0
    avgrec = 0
    count = 0
    avgnmi = 0
    for data in train_loader:
        count = count +1
        data.to(device)
        pred, totloss = model(data.x, data.edge_index)
        pred = pred.max(dim=-1)[1].cpu().detach().numpy()
        pred = pred.squeeze()
        labelsfin = data.y.cpu().detach().numpy()
        avgacc += accuracy_score(labelsfin,pred)
        avgrec += recall(labelsfin,pred)
        avgprec +=precision(labelsfin,pred)
        avgARI +=adjusted_rand_score(labelsfin,pred)
        avgnmi += normalized_mutual_info_score(labelsfin,pred)
    print(f"acc score:{avgacc/count}")
    print(f"recall score:{avgrec/count}")
    print(f"precision score:{avgprec/count}")
    print(f"F1:{2*avgprec*avgrec/(avgprec+avgrec)/count}")
    print(f"ARI:{avgARI/count}")
    print(f"NMI:{avgnmi/count}")
metrics()

Metrics For eval

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import adjusted_rand_score
def metrics():
    model.eval()
    avgacc = 0
    avgARI = 0
    loss_all = 0
    avgprec = 0
    avgrec = 0
    count = 0
    avgnmi = 0
    for data in test_loader:
        count = count +1
        data.to(device)
        pred = model(data.x, data.edge_index, data.edge_weight)
        pred = pred.max(dim=-1)[1].cpu().detach().numpy()
        pred = pred.squeeze()
        labelsfin = data.y.cpu().detach().numpy()
        avgacc += accuracy_score(labelsfin,pred)
        avgrec += recall(labelsfin,pred)
        avgprec +=precision(labelsfin,pred)
        avgARI +=adjusted_rand_score(labelsfin,pred)
        avgnmi += normalized_mutual_info_score(labelsfin,pred)
    print(f"acc score:{avgacc/count}")
    print(f"recall score:{avgrec/count}")
    print(f"precision score:{avgprec/count}")
    print(f"F1:{2*avgprec*avgrec/(avgprec+avgrec)/count}")
    print(f"ARI:{avgARI/count}")
    print(f"NMI:{avgnmi/count}")
metrics()
